In [0]:
import os
import yaml

config_path = os.path.join(os.path.dirname(os.getcwd()), "config.yaml")
with open(config_path, "r") as f:
    config_data = yaml.safe_load(f)

In [0]:
catalog = config_data["databricks_config"]["catalog"]
schema = config_data["databricks_config"]["schema"]
spark.sql(f"USE CATALOG {catalog};")
spark.sql(f"USE SCHEMA {schema}")

In [0]:
spark.sql(
    """
    CREATE OR REPLACE FUNCTION row_filter_by_client(row_client_id STRING)
    RETURNS BOOLEAN
    LANGUAGE SQL
    DETERMINISTIC
    COMMENT 'Row filter: returns TRUE if row belongs to current user'
    RETURN (
        row_client_id = (
            SELECT client_id 
            FROM identity_mappings
            WHERE username = current_user()
            LIMIT 1
        )
    );
    """
)

In [0]:
spark.sql(
    """
    ALTER TABLE customer_transactions
    SET ROW FILTER row_filter_by_client ON (client_id);
    """
)

In [0]:
display(spark.sql("""
          DESCRIBE TABLE EXTENDED customer_transactions;
          """))

In [0]:
display(spark.sql("SELECT * FROM customer_transactions"))